# Measure contributions of parts of OCR pipeline

## The current OCR ensemble pipeline

![ocr_flow](assets/ocr_flow.jpg)

#### The are 4 steps to the full pipeline. The purpose of this notebook is to test if all of these steps really helps with the OCR results, and if they do, by how much.

These steps are currently:
1. Three image pre-processing pipelines: `deskew`, `binarize`, & `denoise` often help with OCR results. Which ones, if any, do we need?
1. Two OCR engines: `Tesseract` & `EasyOCR`. `Tesseract` is the current leader in open source OCR engines nut, does adding `EasyOCR` improve the results?
1. The `combine text` function is only needed if we stick with the ensemble approach. I.e. only if we use more than one image processing pipeline or more than one OCR engine.
1. The `clean text` function corrects misspellings and common OCR errors with punctuation and spacing.

## Comparison strategy

#### We're doing ablations on the OCR pipeline.
- How well do `Tesseract` and `EasyOCR` perform on their own without image pre-processing. I'll also try the engine directly grafted to the `clean text` function.
- How well do each of the image pre-processing steps help the OCR process? and which ones work well with which OCR engine. I'm going to try various permutations of these.
- Can I whittle this down to one or zero image pre-processing pipelines and one OCR engine? If so, then this would allow me to drop the `combine text` step.
- How much does the `clean text` actually step help?

#### Scoring
- I'll use an expert derived gold standard to compare against the ablation sequences.
- We are using Levenshtein distance as the scores. Levenshtein distance counts character mismatches between sequences.

## Setup

In [13]:
from itertools import groupby
from pathlib import Path
from types import SimpleNamespace

import pandas as pd
from tqdm import tqdm

from digi_leap.pylib import consts
from digi_leap.pylib.db import db
from digi_leap.pylib.ocr import ocr_compare as comp

## Constants

In [2]:
ARGS = SimpleNamespace(
    database=Path(consts.DATA_DIR / "sernec" / "sernec.sqlite"),
    gold_set="test_gold_set",
    score_set="notebook_scores",
    notes="",
    csv_path=consts.DATA_DIR / "sernec" / "gold_std_2022-06-28_sample.csv",
)

## Save a gold standard to the database

In [3]:
# comp.new_gold_std(ARGS.csv_path, ARGS.database, ARGS.gold_set)

## Get a gold standard

In [4]:
GOLD_STD = comp.get_gold_std(ARGS.database, ARGS.gold_set)
# gold_std[:2]

## Score the image processing OCR engine combinations

In [5]:
def get_scores(args, gold_std):
    combos = comp.pipeline_engine_combos()

    scores = []

    for gold in tqdm(gold_std):
        images = comp.process_images(gold)
        all_frags = comp.get_ocr_fragments(images, gold)

        scores += comp.score_simple_ocr(args, gold, images)
        scores += comp.score_ocr_ensembles(args, gold, all_frags, combos)

    return scores

In [6]:
SCORES = get_scores(ARGS, GOLD_STD)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:51<00:00, 27.80s/it]


## Save scores

In [10]:
df = pd.DataFrame(SCORES)
with db.connect(ARGS.database) as cxn:
    db.execute(cxn, "delete from ocr_scores where score_set = ?", (ARGS.score_set,))
    df.to_sql("ocr_scores", cxn, if_exists="append", index=False)

## Peek at the scores

In [18]:
grouped_scores = groupby(SCORES, key=lambda s: s["label_id"])

for labels_id, scores in grouped_scores:
    print("=" * 80)
    print(labels_id)
    scores = sorted(scores, key=lambda s: (s['score'], len(s['actions'])))
    for score in scores:
        print(f"{score['score']:4d}  {score['actions']}")

71341
   7  ('', 'easyocr') ('', 'tesseract') ('deskew', 'easyocr') ('deskew', 'tesseract') post_process
   8  ('', 'tesseract') ('deskew', 'tesseract') ('denoise', 'tesseract') post_process
   8  ('', 'tesseract') ('binarize', 'tesseract') ('denoise', 'tesseract') post_process
   8  ('', 'easyocr') ('', 'tesseract') ('binarize', 'easyocr') ('binarize', 'tesseract') post_process
   8  ('deskew', 'easyocr') ('deskew', 'tesseract') ('denoise', 'easyocr') ('denoise', 'tesseract') post_process
   8  ('', 'easyocr') ('', 'tesseract') ('deskew', 'easyocr') ('deskew', 'tesseract') ('binarize', 'easyocr') ('binarize', 'tesseract') post_process
   9  ('', 'easyocr') ('deskew', 'easyocr') ('denoise', 'easyocr') post_process
   9  ('', 'easyocr') ('deskew', 'easyocr') ('binarize', 'easyocr') post_process
   9  ('', 'tesseract') ('deskew', 'tesseract') ('binarize', 'tesseract') ('denoise', 'tesseract') post_process
   9  ('deskew', 'easyocr') ('deskew', 'tesseract') ('binarize', 'easyocr') ('binar